In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

In [2]:
!conda list scipy

# packages in environment at C:\ProgramData\Anaconda3\envs\DataScienceRetreat:
#
# Name                    Version                   Build  Channel
scipy                     1.1.0            py36h4f6bf74_1  


In [3]:
import os
print("Orig Dir:",os.getcwd())
os.chdir('../../')
print("Current Dir:",os.getcwd())

Orig Dir: C:\Users\p.schambach\Desktop\DSR\drone_steering\models\playground
Current Dir: C:\Users\p.schambach\Desktop\DSR\drone_steering


In [4]:
#C:\Users\p.schambach\Desktop\DSR\drone_steering\data\archive\gesture_original\features_flip_c_03_120.csv
data_path = 'data/archive/gesture_original/features_flip_c_03_120.csv'
df_orig = pd.read_csv(data_path,sep=',')

In [5]:
from app_local.module import DataFrameInterpolator

In [6]:
di = DataFrameInterpolator()
di.display_information()
new_df = di.get_new_df(df_orig, 37)
di.display_information()

FrameLength in Milliseconds: 37 

Original Time:
	Number of Frames: 515
	 [  0. 121. 245. 368. 487.] ... [63125. 63254. 63377. 63504. 63629.] 

New Time:
	Number of Frames: 1721
	 [  0  37  74 111 148] ... [63492 63529 63566 63603 63640] 

Original DataFrame Shape: (515, 18)
New DataFrame Shape: (1721, 17)
